In [5]:
from matplotlib import pyplot as plt
# multi-step encoder-decoder lstm 
import tensorflow as tf
from numpy import array
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

size_grid = 6

#Load the pre dataset
act_data1 = []  #human grid pixel change data
point_changes = []  #temporal pixel changes place for one video
for i in range(0, size_grid):
	act_data1.append([])    
	point_changes.append([])

video_index = -1
f = open('clap3.txt', mode = 'rt')

while True:
	line = f.readline()
    #for the last point_changes, one last for and then break while
	if not line: 
		for k in range(0, size_grid):
			if len(point_changes[k]) == 3 :  #should be 2 or more to make better result without one-change-sample               
				act_data1[k].append(point_changes[k])  #one set of change per grid point in total act_data 
		break
	numbers = line.split(' ')  #1 1 : 0 0 0 0 0 0 0 (ex)

	if numbers[0] != video_index :
		print(numbers[0]+'th line get')        
		video_index = numbers[0]        
		if video_index != -1:
			for k in range(0, size_grid):
				if len(point_changes[k]) < 3 :  #should be 2 or more to make better result without one-change-sample
					continue 
				act_data1[k].append(point_changes[k])  #one set of change per grid point in total act_data 
				point_changes[k] = []  #then reset  
             
        #add one value to each grid point change
	for i in range(0, size_grid):
		if len(point_changes[i]) == 3:
			act_data1[i].append(point_changes[i])
			point_changes[i] = []            
		val_int = int(numbers[i+3])        
		point_changes[i].append(val_int)
       
f.close()
print("Dataset load finished")    

1th line get
2th line get
3th line get
4th line get
5th line get
7th line get
8th line get
9th line get
10th line get
11th line get
12th line get
13th line get
14th line get
15th line get
16th line get
17th line get
18th line get
21th line get
22th line get
23th line get
24th line get
25th line get
26th line get
27th line get
30th line get
31th line get
32th line get
33th line get
34th line get
35th line get
36th line get
39th line get
40th line get
41th line get
42th line get
43th line get
44th line get
45th line get
46th line get
47th line get
48th line get
49th line get
50th line get
51th line get
52th line get
54th line get
55th line get
56th line get
57th line get
58th line get
59th line get
60th line get
61th line get
62th line get
63th line get
64th line get
65th line get
66th line get
67th line get
68th line get
69th line get
70th line get
71th line get
72th line get
73th line get
74th line get
75th line get
76th line get
77th line get
78th line get
79th line get
80th line get


In [6]:
           
for j in range(0, size_grid):
    #define dataset
    mid = int(len(act_data1[j])/2)
    X = array(act_data1[j][:mid])
    y = array(act_data1[j][-mid:])
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], 1))
    # define model
    model = tf.keras.Sequential()
    model.add(layers.LSTM(50, activation='relu', input_shape=(3, 1)))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=100, verbose=0)
    #and save
    print("%d th model finished" % j)
    name = str('C:/Users/zoclz/models/6model%d.h5'%j)
    model.save(name)
    del model

print("------------All train finished---------------")

0 th model finished
1 th model finished
2 th model finished
3 th model finished
4 th model finished
5 th model finished
------------All train finished---------------
